In [16]:

login_url = 'https://secure.indeed.com/account/login'

In [17]:
#type in your indeed account and password
data = {
        'action':'login',
        '__email':'*****',
        '__password':'******',
        'remember':'1',
        'hl':'en',
        'continue':'/account/view?hl=en',
       }

In [18]:
header = {'User_Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'}

In [19]:
def periodfinder(content,class_info):
    if content.find('p',class_=class_info) is not None:
            period = i.find('p',class_=class_info).text.split(' to ')
            #print (period)   # AA BB 1, AA PRESENT 2 A PRESENT 3 A BB 4 AA B 5
            if len(period) == 2:
                if period[1] == 'Present':
                    try:
                        period[1] = datetime.today().strftime('%B %Y')
                    except:
                        period[1] = datetime.today().strftime('%b %Y')
                if len(period[0]) > 4 and len(period[1]) > 4:
                    #type AA BB
                    try:
                        prd = (datetime.strptime(period[1],'%B %Y').date()-\
                        datetime.strptime(period[0],'%B %Y').date())
                    except:
                        prd = (datetime.strptime(period[1],'%b %Y').date()-\
                        datetime.strptime(period[0],'%b %Y').date())
                    
                    YOE = int(abs(round(prd.days/365,0)))
                elif len(period[0]) > 4 and len(period[1]) < 5:
                    #type AA B
                    YOE = int(period[1])-int(period[0].split()[1])
                elif len(period[0]) < 5 and len(period[1]) > 4:
                    #type A BB
                    YOE = int(period[1].split()[1])-int(period[0])
                elif len(period[0]) < 5 and len(period[1]) < 5:
                    #type A B
                    YOE = int(period[1])-int(period[0])
            else:
                YOE = 1
            if YOE < 1:
                YOE = 'less than 1 year'
    else:
        YOE = 'No lenth info'
    return YOE

In [20]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [21]:
s = requests.session()

#login with login and password

In [22]:
r = s.post(login_url,data = data)

In [23]:
#get resume page1 

rq= s.get('https://www.indeed.com/resumes?q=Data+Scientist&l=GTA%2C+ON')
url = []

In [24]:
soup = ''

In [25]:
all_content=[]
soup = BeautifulSoup(rq.text,'html.parser')
content = soup.find_all('a', class_= 'app_link')

In [26]:
for c in content:
    full = 'https://www.indeed.com'+c.attrs['href']
    url.append(full)
cnt = len(content)
#open each url and look for contents





In [27]:
import pandas as pd
import nltk
import random

In [28]:
columns = ["resume_link","Employer","Job_Desc","Num_of_Exp"]
df_resume = pd.DataFrame(columns = columns)

In [29]:
file = {}

In [30]:
for j in range(0,cnt):
    
    resume = []

    r = s.get(url[j],headers = header)
    
    soup = BeautifulSoup(r.text,'html')
    
    #rename names
    
    name = soup.find('h1').text
    
    print(name)
    
    

    experience = soup.find_all('div',class_ = re.compile(r'work-experience-section( \w)*'))
    
    for i in experience:
        num = (len(df_resume)+1)
        resume.append(url[j])
        
        if i.find('p',class_='work_title title') is None:
            jobtitle = 'No Job Title'
        else:
            jobtitle = i.find('p',class_='work_title title').text
        if i.find('div',class_='work_company') is None:
            comp_lo = 'No Company'
        else:
            comp_lo = i.find('div',class_='work_company').text.split(' - ')
        if len(comp_lo) == 2:
            company,location = i.find('div',class_='work_company').text.split(' - ')
        elif len(comp_lo) ==1 :
            company = i.find('div',class_='work_company').text
        else:
            company = 'No Company'
        resume.append(company)
        if i.find('p',class_='work_dates') is not None:
            YOE = 0
            period = i.find('p',class_='work_dates').text.split(' to ')
            #print (period)   # AA BB 1, AA PRESENT 2 A PRESENT 3 A BB 4 AA B 5
            if len(period) == 2:
                if period[1] == 'Present':
                    try:
                        period[1] = datetime.today().strftime('%B %Y')
                    except:
                        period[1] = datetime.today().strftime('%b %Y')
                if len(period[0]) > 4 and len(period[1]) > 4:
                    #type AA BB
                    try:
                        prd = (datetime.strptime(period[1],'%B %Y').date()-\
                        datetime.strptime(period[0],'%B %Y').date())
                    except:
                        prd = (datetime.strptime(period[1],'%b %Y').date()-\
                        datetime.strptime(period[0],'%b %Y').date())
                    
                    YOE = int(abs(round(prd.days/365,0)))
                elif len(period[0]) > 4 and len(period[1]) < 5:
                    #type AA B
                    YOE = int(period[1])-int(period[0].split()[1])
                elif len(period[0]) < 5 and len(period[1]) > 4:
                    #type A BB
                    YOE = int(period[1].split()[1])-int(period[0])
                elif len(period[0]) < 5 and len(period[1]) < 5:
                    #type A B
                    YOE = int(period[1])-int(period[0])
            else:
                YOE = 1
            if YOE < 1:
                YOE = 'less than 1 year'
        else:
            YOE = 'No lenth of work info'
        resume.append(YOE)
        jobdesc = i.find('p',class_='work_description')
        
        if jobdesc is None:
            desc = ''
        else:
            desc = jobdesc.text
        
        resume.append(desc)
        
        print(jobtitle,'\n',company,location,'\n',desc,'\n',YOE,'year')
    
    #file[name] = resume  
    #break

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Data Science Research Assistant
Data Science Research Assistant 
 Memorial University of Newfoundland St. John's, NL 
 --This topic of program was survival analysis in high dimension feature space. The goal of the program was to analyze and identify subtypes of lung cancer survival data, which has high dimension gene feature spaces, survival time and censoring statues indicating if the survival time is completed. --Read literatures, carried out research on the algorithms identifying subtypes of survival data. --Utilized R to design, build, and implement semi-supervised models, the models based on unsupervised learning, and the models based on clinical data, to identity subtypes of lung cancer survival data. --Focused on optimizing models. Optimized the intermediate results and found the best gene set has around 20 genes, which matched quite well with the literatures. Discovered some weak points and enabled each predictive model to make better classification. --Created a new version of 

DATA SCIENTIST
DATA SCIENTIST 
 BDI Inc Los Gatos, CA 
 Business: Consulting  • Working in a Data Scientist capacity on a Machine/Deep Learning project privately. 
 1 year
DATA ARCHITECT/DATA MODELER 
 Great West Life Los Gatos, CA 
 Business: Insurance/Investment • Key Projects: * Configuration and Asset Management Project of the corporation's IT Infrastructure Domain using Remedy/ITIL * Configuration Management Database (CMDB) Conceptual Data Model • Worked as Data Architect in the areas of Data Modeling, Data Architecture and Data Governance in an Agile environment. • Developed a Configuration Management Database (CMDB) Conceptual Data Model and reports. • Created about a dozen Subject Area Logical Data Models and about 10 Assets Logical Data Models and reports. • Created Configuration Item (CI) Relationship and Data Federation Conceptual Data Models and reports. • Created Data Architecture and Data Governance frameworks for the project and documentations. •  Technologies include BM

Data Scientist
Data Scientist 
 CentricIQ Markham, ON 
 • Delivered multiple projects implementing all phases of data science life cycle, including data processing, data exploration, feature engineering, predictive model, implementation and validation with large structured and unstructured data • Evaluated and analyzed customer end to end Omni-Channel journey both online and offline across multiple channels and touch points • Explored, analyzed and characterized demographic, psychographic and behavioral data of approx. 244M US population and 1300 personal attributes to identify potential customers • Identified key market segmentation and analyzed trends and opportunities in large and rich data sets across multiple dimensions to drive client marketing strategies • Developed and implemented predictive & statistical models using Machine learning techniques such as Clustering, Regression models, classification, Decision Trees, Random Forest to target prospects with higher probability to re

Data Scientist
Data Scientist 
 HouseSigma Toronto, ON 
 ●    Implementing the job to do Text Mining using Python’s libraries  ‒    Further processing the dataset of the GTA houses describing features in the company’s database saved in MySQL.  ‒    Using NLP tools like NLTK, Spacy, Word2vec, Sense2vec to process the reviews of customers, i.e., tokenization, lemmatization, and clustering for building deep learning model of predicting housing price.             ▪              ▪ Data cleaning, representation, visualization and transformation for processing more than 200000 data features.             ▪ Building regression models of KNN, Lasso, Ridge, and Neural Network for houses renting prices 
 less than 1 year year
Teaching Assistant 
 Electrical and Computing Fundamental Toronto, ON 
 Assist in helping the supervisor of Electrical and Computing Fundamental to conduct the students do the physical and computer labs. Also, to help the teacher mark the reports and some exams. 
 1 year
Rese

...

Environmental Scientist
Environmental Scientist 
 Terrapex Environmental Ltd Toronto, ON 
 Managed and executed many environmental projects.   Performed Phase I Environmental Site Assessments (ESAs) which include: historical research, data collection and interpretation, site inspections, report writing and client liaison  Performed Phase II Environmental Site Assessments (ESAs) field work which include: surveying using level and rod, drilling boreholes and test pitting operations, excavations, tank removal, hoist removal, methane sampling, Designated Substance Surveys, soil logging and sampling, installation of monitoring wells, ground water & vapour sampling, silica vapour sampling, sewer inspection   Other task include : analyse, review and tabulate field data , report writing, autoCAD submissions and review, client/contractor liaison, invoice/budget tracking and management, scheduling/coordinating work, ordering and maintaining equipment, and coordinate laboratory sample pick up and

Machine Learning scientist
Machine Learning scientist 
 5M Capital investment Köln 
 Focus:  Large data analysis, pattern recognition, supervised and unsupervised learning, Deep Learning, Reinforcement Learning, data analysis, FOREX market prediction and algotrading.  Reinforcement Learning, Deep learning, Recurrent Networks, are used to integrate the Machine Learning techniques with traditional analytical trading techniques which enables the algotrading platform to learn in real time and predict the market based on learned patterns. Several different statistical data modeling are also applied to model the market behavior and patterns. 
 3 year
Research Assistant 
 PyTorch, Tenserflow, Keras Sherbrooke, QC 
 Focus: Computer Vision, Machine Learning (Deep learning, Autoencoders, Convolutional neural  networks), Object detection and localization, fast data and image retrieval. ● Achievements:  - Supervised and unsupervised learning notworks are used for object lozalization and image rank

Pharmaceutical Scientist
Pharmaceutical Scientist 
 COVAR PHARMACEUTICALS INC Mississauga, ON 
 Formulation, pre-formulation; design, coordinate, and execute experiments for clinical trial phase 1 and 2 projects; IND filing; technology transfer; wrote pharmaceutical processing equipment SOPs; equipment IQ/OQ/PQ; equipment sourcing and purchasing. •   Developed analytical methods (mainly UV, HPLC, Dissolution); method transfer; method validation; analytical equipment set-up and experiments; chemical sourcing and purchasing; analytical equipment sourcing and purchasing. Performed analytical tests, documented results, and generated reports according to GMP requirements. •   Understood quality system and wrote SOPs relate to validations, cleaning verification etc. •   GMP high potency facility set-up based on Ontario EH&S, Health Canada and FDA requirements; SOP writing; developed filing database and set-up company cloud central filing system; license acquiring; submission to FDA and Healt

Environmental Scientist
Environmental Scientist 
 Arcadis Canada Inc. Mississauga, ON 
 • Completed various biological and ecological studies through field surveys and desktop reviews in compliance with provincial and federal Environmental Assessments. • Executed several surface water and ground water investigations in various land-use settings in support of Phase I and  II Environmental Site Assessments, and long-term sampling and monitoring projects. • Performed numerous biological surveys including: vegetation, ecological land classification in support, species at risk, breeding birds, and amphibians across terrestrial ecosystems. • Supported subsurface soil investigations through borehole, excavation, and test-pit sampling in determining the extent and magnitude of potential contaminants. • Suggested multiple tree and vegetation protection, mitigation, and compensation plans prior to construction works within woodlots and forests under acts and regulations related to forestry opera

Scientist I
Scientist I 
 VALE Toronto, ON 
 Electron microscopist and research scientist with focus on material characterization and material extraction via thermal processing.  • Collaborated with staff and clients to deliver high quality and comprehensive studies to include material preparation, analysis, data processing and interpretation creating an efficient and relevant material characterization study using the following techniques: o Scanning electron microscopy (SEM) - secondary electron (SEI) and backscattered imaging (BSE) (JEOL JSM-7000F & JEOL 6400) o Qualitative and quantitative energy dispersive x-ray spectroscopy (EDS) o X-ray diffraction - pattern analysis and data evaluation (Bruker D8 Advance) - phase identification and Rietveld quantification o Optical microscopy - study physical structure, topography, metallography, imaging, mineralogical phase identification • Generated detailed reports highlighting results from material characterization studies associated with in

In [31]:
file[""]

KeyError: ''